In [1]:
import requests
from bs4 import BeautifulSoup
from google.colab import files
import csv
import os
import re
import pandas as pd

In [2]:
DOMAIN = 'https://www.nfl.com/stats/player-stats/category'

# Lista para armazenar as URLs encontradas
URLs = [ f'{DOMAIN}/fumbles/2024/post/all/defensiveforcedfumble/desc',
        f'{DOMAIN}/fumbles/2024/REG/all/defensiveforcedfumble/DESC?aftercursor=AAAAGQAAABhAAAAAAAAAADFleUp6WldGeVkyaEJablJsY2lJNld5SXlJaXdpTXpJd01EUXhOVEl0TkdReU1TMDRPVEExTFdaaFlXRXRNMlkwWkRrME16QTNaRFl3SWl3aU1qQXlOQ0pkZlE9PQ==',
        f'{DOMAIN}/fumbles/2024/REG/all/defensiveforcedfumble/DESC?aftercursor=AAAAMgAAABhAAAAAAAAAADFleUp6WldGeVkyaEJablJsY2lJNld5SXlJaXdpTXpJd01EUmhOR1l0TkdVMU55MDVPVE00TFdFMU5UY3RPV0U0TlRrNU16QmpNV0ppSWl3aU1qQXlOQ0pkZlE9PQ==',
        f'{DOMAIN}/fumbles/2024/REG/all/defensiveforcedfumble/DESC?aftercursor=AAAASwAAABhAAAAAAAAAADFleUp6WldGeVkyaEJablJsY2lJNld5SXlJaXdpTXpJd01EVTJOREV0TkdVMk5TMDJOakEwTFRNMVpXSXROemt3T1RReE5XVTBNVFUxSWl3aU1qQXlOQ0pkZlE9PQ==',
        f'{DOMAIN}/fumbles/2024/REG/all/defensiveforcedfumble/DESC?aftercursor=AAAAZAAAAFM_8AAAAAAAADFleUp6WldGeVkyaEJablJsY2lJNld5SXhJaXdpTXpJd01EUXlORGt0TlRNeU5DMDBPVFUxTFRjNE1qZ3RZMlppT1dNd1lUWmxZbUZsSWl3aU1qQXlOQ0pkZlE9PQ==',
        f'{DOMAIN}/fumbles/2024/REG/all/defensiveforcedfumble/DESC?aftercursor=AAAAfQAAAFM_8AAAAAAAADFleUp6WldGeVkyaEJablJsY2lJNld5SXhJaXdpTXpJd01EUXpOR1l0TkdVME5pMHhPVGs1TFRVNVpUa3RNalV6WTJWa1pXVmpNMkUxSWl3aU1qQXlOQ0pkZlE9PQ==',
        f'{DOMAIN}/fumbles/2024/REG/all/defensiveforcedfumble/DESC?aftercursor=AAAAlgAAAFM_8AAAAAAAADFleUp6WldGeVkyaEJablJsY2lJNld5SXhJaXdpTXpJd01EUTJOR010TkdZM05TMDRNRFl3TFRWaE5tUXRaRE5oWVdKak0yTXpNbVV5SWl3aU1qQXlOQ0pkZlE9PQ==',
        f'{DOMAIN}/fumbles/2024/REG/all/defensiveforcedfumble/DESC?aftercursor=AAAArwAAAFM_8AAAAAAAADFleUp6WldGeVkyaEJablJsY2lJNld5SXhJaXdpTXpJd01EUTROVFV0TkRZd09TMHdPRFEwTFRZNFlqRXROall4TnpRME1XVmhZVFUzSWl3aU1qQXlOQ0pkZlE9PQ==',
        f'{DOMAIN}/fumbles/2024/REG/all/defensiveforcedfumble/DESC?aftercursor=AAAAyAAAAFM_8AAAAAAAADFleUp6WldGeVkyaEJablJsY2lJNld5SXhJaXdpTXpJd01EUmpORFV0TlRjME1TMDVOVGszTFRZMllXSXRZakU0WkRWbVlqWTFPV0UxSWl3aU1qQXlOQ0pkZlE9PQ==',
        f'{DOMAIN}/fumbles/2024/REG/all/defensiveforcedfumble/DESC?aftercursor=AAAA4QAAAFM_8AAAAAAAADFleUp6WldGeVkyaEJablJsY2lJNld5SXhJaXdpTXpJd01EUmxORFV0TlRjM01TMDBPRGcxTFRneVl6Y3RORGs1TURZeE1HTTNZMlUzSWl3aU1qQXlOQ0pkZlE9PQ==',
        f'{DOMAIN}/fumbles/2024/REG/all/defensiveforcedfumble/DESC?aftercursor=AAAA-gAAAFM_8AAAAAAAADFleUp6WldGeVkyaEJablJsY2lJNld5SXhJaXdpTXpJd01EVXlORFV0TkRFMk5pMDBNVFV4TFdJM04yUXROREptT1RabE0yWTBZekUzSWl3aU1qQXlOQ0pkZlE9PQ==',
        f'{DOMAIN}/fumbles/2024/REG/all/defensiveforcedfumble/DESC?aftercursor=AAABEwAAAFM_8AAAAAAAADFleUp6WldGeVkyaEJablJsY2lJNld5SXhJaXdpTXpJd01EVXpOVFF0TkdFMk1TMDFNekV3TFdabU9ERXRaVFUzWlRrNU16QTJZVEk1SWl3aU1qQXlOQ0pkZlE9PQ==',
        f'{DOMAIN}/fumbles/2024/REG/all/defensiveforcedfumble/DESC?aftercursor=AAABLAAAAFM_8AAAAAAAADFleUp6WldGeVkyaEJablJsY2lJNld5SXhJaXdpTXpJd01EVTNORGt0TkRjeE9DMDNNVEUyTFRrMFptSXROV015WW1GbVlqY3pZMlkxSWl3aU1qQXlOQ0pkZlE9PQ==',
        f'{DOMAIN}/fumbles/2024/REG/all/defensiveforcedfumble/DESC?aftercursor=AAABRQAAATsAAAAAAAAAADFleUp6WldGeVkyaEJablJsY2lJNld5SXdJaXdpTXpJd01EUXhORFF0TkRFMU1TMDVPVEExTFRjd1pESXRPVEk0TnpjeU5UZGlNRFkwSWl3aU1qQXlOQ0pkZlE9PQ==',
        f'{DOMAIN}/fumbles/2024/REG/all/defensiveforcedfumble/DESC?aftercursor=AAABXgAAATsAAAAAAAAAADFleUp6WldGeVkyaEJablJsY2lJNld5SXdJaXdpTXpJd01EUXhOR1V0TkRjeU9DMHdPVEV4TFdOaFpESXRaVGhqTjJOaE5ETTBPVFUySWl3aU1qQXlOQ0pkZlE9PQ==',
        f'{DOMAIN}/fumbles/2024/REG/all/defensiveforcedfumble/DESC?aftercursor=AAABdwAAATsAAAAAAAAAADFleUp6WldGeVkyaEJablJsY2lJNld5SXdJaXdpTXpJd01EUXlOREV0TlRJMU1pMDRNekU0TFdRNU1UWXRNMlJqWVRjMVlqUmhabUZpSWl3aU1qQXlOQ0pkZlE9PQ==',
        f'{DOMAIN}/fumbles/2024/REG/all/defensiveforcedfumble/DESC?aftercursor=AAABkAAAATsAAAAAAAAAADFleUp6WldGeVkyaEJablJsY2lJNld5SXdJaXdpTXpJd01EUXlORFV0TlRJM055MDVOVGc0TFdFd05XVXROakZpWldZek5qRmxZemxqSWl3aU1qQXlOQ0pkZlE9PQ==',
        f'{DOMAIN}/fumbles/2024/REG/all/defensiveforcedfumble/DESC?aftercursor=AAABqQAAATsAAAAAAAAAADFleUp6WldGeVkyaEJablJsY2lJNld5SXdJaXdpTXpJd01EUXlOR1l0TlRjM055MHdNREl5TFdaaE9HTXRZakU0WTJOaE0ySmtOMlkySWl3aU1qQXlOQ0pkZlE9PQ==',
        f'{DOMAIN}/fumbles/2024/REG/all/defensiveforcedfumble/DESC?aftercursor=AAABwgAAATsAAAAAAAAAADFleUp6WldGeVkyaEJablJsY2lJNld5SXdJaXdpTXpJd01EUXlOVEl0TkdZMU5pMDBPVFEyTFRaa1pXTXRNelpoTnpNM1lqTXlPV1UwSWl3aU1qQXlOQ0pkZlE9PQ==',
        f'{DOMAIN}/fumbles/2024/REG/all/defensiveforcedfumble/DESC?aftercursor=AAAB2wAAATsAAAAAAAAAADFleUp6WldGeVkyaEJablJsY2lJNld5SXdJaXdpTXpJd01EUXpOREV0TkdReU1pMDNPVEl5TFdaaVpXVXRZVGhoWmpZNE5USTRNbVV6SWl3aU1qQXlOQ0pkZlE9PQ==',
        f'{DOMAIN}/fumbles/2024/REG/all/defensiveforcedfumble/DESC?aftercursor=AAAB9AAAATsAAAAAAAAAADFleUp6WldGeVkyaEJablJsY2lJNld5SXdJaXdpTXpJd01EUXpOR010TkRFME1DMDVNalV4TFRKbU9EZ3RaRFExTkdJNFlXVTJNek5pSWl3aU1qQXlOQ0pkZlE9PQ==',
        f'{DOMAIN}/fumbles/2024/REG/all/defensiveforcedfumble/DESC?aftercursor=AAACDQAAATsAAAAAAAAAADFleUp6WldGeVkyaEJablJsY2lJNld5SXdJaXdpTXpJd01EUXpOR1l0TlRnMU1pMDBPRGd3TFdOa05HWXROV1U0WkdabFlqWmhaVFpqSWl3aU1qQXlOQ0pkZlE9PQ==',
        f'{DOMAIN}/fumbles/2024/REG/all/defensiveforcedfumble/DESC?aftercursor=AAACJgAAATsAAAAAAAAAADFleUp6WldGeVkyaEJablJsY2lJNld5SXdJaXdpTXpJd01EUTBOREV0TlRZMU1TMHpPVGc1TFRjek5URXROV1ZsWVRZM01UazNZelkzSWl3aU1qQXlOQ0pkZlE9PQ==',
        f'{DOMAIN}/fumbles/2024/REG/all/defensiveforcedfumble/DESC?aftercursor=AAACPwAAATsAAAAAAAAAADFleUp6WldGeVkyaEJablJsY2lJNld5SXdJaXdpTXpJd01EUTBOR1l0TlRVeU5TMDVOVFV5TFRWa1l6a3RabVJoTjJNMVlUbGlNRE5rSWl3aU1qQXlOQ0pkZlE9PQ==',
        f'{DOMAIN}/fumbles/2024/REG/all/defensiveforcedfumble/DESC?aftercursor=AAACWAAAATsAAAAAAAAAADFleUp6WldGeVkyaEJablJsY2lJNld5SXdJaXdpTXpJd01EUTFOR010TlRVeE55MDNPVGd4TFRKa1lqTXRNamt5WVRNd1pqUTVNMlV5SWl3aU1qQXlOQ0pkZlE9PQ==',
        f'{DOMAIN}/fumbles/2024/REG/all/defensiveforcedfumble/DESC?aftercursor=AAACcQAAATsAAAAAAAAAADFleUp6WldGeVkyaEJablJsY2lJNld5SXdJaXdpTXpJd01EUTJOR010TkdZd01DMHdNREF3TFdZeE5qUXRPREUzTnpsbU9XSTRORGd5SWl3aU1qQXlOQ0pkZlE9PQ==',
        f'{DOMAIN}/fumbles/2024/REG/all/defensiveforcedfumble/DESC?aftercursor=AAACigAAATsAAAAAAAAAADFleUp6WldGeVkyaEJablJsY2lJNld5SXdJaXdpTXpJd01EUTNOREV0TkRrMk1DMDJPVFkwTFRjM01ESXROamt5TmpVMlptRmtPRFl4SWl3aU1qQXlOQ0pkZlE9PQ==',
        f'{DOMAIN}/fumbles/2024/REG/all/defensiveforcedfumble/DESC?aftercursor=AAACowAAATsAAAAAAAAAADFleUp6WldGeVkyaEJablJsY2lJNld5SXdJaXdpTXpJd01EUTNOR1l0TkRRMU5DMDVNemt6TFRabFpXTXRNalF4T0Rsa1pEWTJOalF4SWl3aU1qQXlOQ0pkZlE9PQ==',
        f'{DOMAIN}/fumbles/2024/REG/all/defensiveforcedfumble/DESC?aftercursor=AAACvAAAATsAAAAAAAAAADFleUp6WldGeVkyaEJablJsY2lJNld5SXdJaXdpTXpJd01EUTNOVFV0TlRreU15MHdOekUyTFRsak1Ua3ROV1ExTWpJeFpqZ3pPV1U0SWl3aU1qQXlOQ0pkZlE9PQ==',
        f'{DOMAIN}/fumbles/2024/REG/all/defensiveforcedfumble/DESC?aftercursor=AAAC1QAAATsAAAAAAAAAADFleUp6WldGeVkyaEJablJsY2lJNld5SXdJaXdpTXpJd01EUTROREV0TlRJME5TMHdOVFF5TFRZME1HSXROV1JqTlRjeU9EUTFZalpqSWl3aU1qQXlOQ0pkZlE9PQ==',
        f'{DOMAIN}/fumbles/2024/REG/all/defensiveforcedfumble/DESC?aftercursor=AAAC7gAAATsAAAAAAAAAADFleUp6WldGeVkyaEJablJsY2lJNld5SXdJaXdpTXpJd01EUTRORGt0TkRNME5pMDFPREl5TFRkbU9USXRObUppTm1FelkyRTJNR1UwSWl3aU1qQXlOQ0pkZlE9PQ==',
        f'{DOMAIN}/fumbles/2024/REG/all/defensiveforcedfumble/DESC?aftercursor=AAADBwAAATsAAAAAAAAAADFleUp6WldGeVkyaEJablJsY2lJNld5SXdJaXdpTXpJd01EUTROR1l0TkdZeU9TMDROelV6TFRZNVptTXROVFppTmpoaU0yRTVZakUzSWl3aU1qQXlOQ0pkZlE9PQ==',
        f'{DOMAIN}/fumbles/2024/REG/all/defensiveforcedfumble/DESC?aftercursor=AAADIAAAATsAAAAAAAAAADFleUp6WldGeVkyaEJablJsY2lJNld5SXdJaXdpTXpJd01EUTVOR1V0TkRjMU5DMDFORE16TFRZM01ESXROV0kwWXpVNVlXTmhNMkU0SWl3aU1qQXlOQ0pkZlE9PQ==',
        f'{DOMAIN}/fumbles/2024/REG/all/defensiveforcedfumble/DESC?aftercursor=AAADOQAAATsAAAAAAAAAADFleUp6WldGeVkyaEJablJsY2lJNld5SXdJaXdpTXpJd01EUmhORFV0TkRZME1pMHpOVE00TFRZeU5EQXRPREZrWXpGa1pqUmpNekpsSWl3aU1qQXlOQ0pkZlE9PQ==',
        f'{DOMAIN}/fumbles/2024/REG/all/defensiveforcedfumble/DESC?aftercursor=AAADUgAAATsAAAAAAAAAADFleUp6WldGeVkyaEJablJsY2lJNld5SXdJaXdpTXpJd01EUmhOR1l0TkRnek5pMHlOalF4TFRrNU1ETXRaamMwTVRkak1URTJPRFUzSWl3aU1qQXlOQ0pkZlE9PQ==',
        f'{DOMAIN}/fumbles/2024/REG/all/defensiveforcedfumble/DESC?aftercursor=AAADawAAATsAAAAAAAAAADFleUp6WldGeVkyaEJablJsY2lJNld5SXdJaXdpTXpJd01EUmhOR1l0TkdVM05pMDROemd5TFRFMk0yTXRPR1E1WkdJeU9XWTFZMkU1SWl3aU1qQXlOQ0pkZlE9PQ==',
        f'{DOMAIN}/fumbles/2024/REG/all/defensiveforcedfumble/DESC?aftercursor=AAADhAAAATsAAAAAAAAAADFleUp6WldGeVkyaEJablJsY2lJNld5SXdJaXdpTXpJd01EUmlORGt0TlRJeU5TMDNPVEU0TFdVME1ESXRPREZtWldWa00ySmtZV1ppSWl3aU1qQXlOQ0pkZlE9PQ==',
        f'{DOMAIN}/fumbles/2024/REG/all/defensiveforcedfumble/DESC?aftercursor=AAADnQAAATsAAAAAAAAAADFleUp6WldGeVkyaEJablJsY2lJNld5SXdJaXdpTXpJd01EUmpOREV0TldFd05DMHpOekUwTFdObVlqY3RPRGN5WWpJd1lqQmpaVEV6SWl3aU1qQXlOQ0pkZlE9PQ==',
        f'{DOMAIN}/fumbles/2024/REG/all/defensiveforcedfumble/DESC?aftercursor=AAADtgAAATsAAAAAAAAAADFleUp6WldGeVkyaEJablJsY2lJNld5SXdJaXdpTXpJd01EUmtOREV0TkdNMk5DMDNORGswTFRnMk1UQXROMkk1TjJJNFpHTmhZV0kxSWl3aU1qQXlOQ0pkZlE9PQ==',
        f'{DOMAIN}/fumbles/2024/REG/all/defensiveforcedfumble/DESC?aftercursor=AAADzwAAATsAAAAAAAAAADFleUp6WldGeVkyaEJablJsY2lJNld5SXdJaXdpTXpJd01EUmtORE10TkRNME9TMDFNVGcxTFRFMlpURXRZV000TW1FeVptTTFOalV4SWl3aU1qQXlOQ0pkZlE9PQ==',
        f'{DOMAIN}/fumbles/2024/REG/all/defensiveforcedfumble/DESC?aftercursor=AAAD6AAAATsAAAAAAAAAADFleUp6WldGeVkyaEJablJsY2lJNld5SXdJaXdpTXpJd01EUmtORFV0TkdNMU55MDFORE14TFRFd05qRXRZak14TkdFME1qYzJNbUptSWl3aU1qQXlOQ0pkZlE9PQ==',
        f'{DOMAIN}/fumbles/2024/REG/all/defensiveforcedfumble/DESC?aftercursor=AAAEAQAAATsAAAAAAAAAADFleUp6WldGeVkyaEJablJsY2lJNld5SXdJaXdpTXpJd01EUmtOR1l0TkdZM01DMDJOVEU0TFdWbU5UQXRORFU1TWpVellXVTJZekF6SWl3aU1qQXlOQ0pkZlE9PQ==',
        f'{DOMAIN}/fumbles/2024/REG/all/defensiveforcedfumble/DESC?aftercursor=AAAEGgAAATsAAAAAAAAAADFleUp6WldGeVkyaEJablJsY2lJNld5SXdJaXdpTXpJd01EUmxORFV0TkRFME55MDFPRFE0TFRrek16Y3RNR1F5WlRZd056STVOamd4SWl3aU1qQXlOQ0pkZlE9PQ==',
        f'{DOMAIN}/fumbles/2024/REG/all/defensiveforcedfumble/DESC?aftercursor=AAAEMwAAATsAAAAAAAAAADFleUp6WldGeVkyaEJablJsY2lJNld5SXdJaXdpTXpJd01EUm1OR0V0TkRFME1TMDFNamt4TFRsak5tUXRPRFE1TWpFM1lUTXhNbUl6SWl3aU1qQXlOQ0pkZlE9PQ==',
        f'{DOMAIN}/fumbles/2024/REG/all/defensiveforcedfumble/DESC?aftercursor=AAAETAAAATsAAAAAAAAAADFleUp6WldGeVkyaEJablJsY2lJNld5SXdJaXdpTXpJd01EVXdOREV0TlRreU9DMHhNakl5TFRaaFptRXROREZsWXpreU5UWXlOMkUwSWl3aU1qQXlOQ0pkZlE9PQ==',
        f'{DOMAIN}/fumbles/2024/REG/all/defensiveforcedfumble/DESC?aftercursor=AAAEZQAAATsAAAAAAAAAADFleUp6WldGeVkyaEJablJsY2lJNld5SXdJaXdpTXpJd01EVXdOR1l0TlRreE55MDNPVGd4TFRRNVpUTXRORFV3TkdReE1ETTJPV1ppSWl3aU1qQXlOQ0pkZlE9PQ==',
        f'{DOMAIN}/fumbles/2024/REG/all/defensiveforcedfumble/DESC?aftercursor=AAAEfgAAATsAAAAAAAAAADFleUp6WldGeVkyaEJablJsY2lJNld5SXdJaXdpTXpJd01EVXlORFV0TkRrek1DMDJOekl6TFdNME5tWXRNRE5rWkdFM05UTXlZV05rSWl3aU1qQXlOQ0pkZlE9PQ==',
        f'{DOMAIN}/fumbles/2024/REG/all/defensiveforcedfumble/DESC?aftercursor=AAAElwAAATsAAAAAAAAAADFleUp6WldGeVkyaEJablJsY2lJNld5SXdJaXdpTXpJd01EVXlOR1l0TkRJek9DMDBPRGc0TFRZMVltSXRZbVEzWVRJek1qRTNaR1F6SWl3aU1qQXlOQ0pkZlE9PQ==',
        f'{DOMAIN}/fumbles/2024/REG/all/defensiveforcedfumble/DESC?aftercursor=AAAEsAAAATsAAAAAAAAAADFleUp6WldGeVkyaEJablJsY2lJNld5SXdJaXdpTXpJd01EVXpOREV0TkdVd05DMDNOell5TFRRM056VXRPVGRsWkdJM056RTVNbVEzSWl3aU1qQXlOQ0pkZlE9PQ==',
        f'{DOMAIN}/fumbles/2024/REG/all/defensiveforcedfumble/DESC?aftercursor=AAAEyQAAATsAAAAAAAAAADFleUp6WldGeVkyaEJablJsY2lJNld5SXdJaXdpTXpJd01EVXpORGd0TkRVNE1DMDJNak01TFRJMU1HSXRaR1poT1RFd1pqWXlOekprSWl3aU1qQXlOQ0pkZlE9PQ==',
        f'{DOMAIN}/fumbles/2024/REG/all/defensiveforcedfumble/DESC?aftercursor=AAAE4gAAATsAAAAAAAAAADFleUp6WldGeVkyaEJablJsY2lJNld5SXdJaXdpTXpJd01EVXpOR1F0TkRrM015MDVNRFF5TFdFeVpUVXROMkUyTVRGak0yTXpNakUwSWl3aU1qQXlOQ0pkZlE9PQ==',
        f'{DOMAIN}/fumbles/2024/REG/all/defensiveforcedfumble/DESC?aftercursor=AAAE-wAAATsAAAAAAAAAADFleUp6WldGeVkyaEJablJsY2lJNld5SXdJaXdpTXpJd01EVXpOVFF0TkRrek9TMDRPRFUyTFRrNFpHTXRORGsyTXpReFlXVXpOVEEySWl3aU1qQXlOQ0pkZlE9PQ==',
        f'{DOMAIN}/fumbles/2024/REG/all/defensiveforcedfumble/DESC?aftercursor=AAAFFAAAATsAAAAAAAAAADFleUp6WldGeVkyaEJablJsY2lJNld5SXdJaXdpTXpJd01EVTBOREV0TlRZek9DMHdNak00TFdRMlkyTXRNalZoWVRFM1lqRTRZekJrSWl3aU1qQXlOQ0pkZlE9PQ==',
        f'{DOMAIN}/fumbles/2024/REG/all/defensiveforcedfumble/DESC?aftercursor=AAAFLQAAATsAAAAAAAAAADFleUp6WldGeVkyaEJablJsY2lJNld5SXdJaXdpTXpJd01EVTBOR1l0TkdNeE5DMDRPVEUwTFdSbE9URXRZMlExTkdRMU9XUmxNbUV5SWl3aU1qQXlOQ0pkZlE9PQ==',
        f'{DOMAIN}/fumbles/2024/REG/all/defensiveforcedfumble/DESC?aftercursor=AAAFRgAAATsAAAAAAAAAADFleUp6WldGeVkyaEJablJsY2lJNld5SXdJaXdpTXpJd01EVTFOVEl0TkRJeU1pMDJOVEl4TFRsbE1UQXRZalV6TlRjell6VmpaR1UwSWl3aU1qQXlOQ0pkZlE9PQ==',
        f'{DOMAIN}/fumbles/2024/REG/all/defensiveforcedfumble/DESC?aftercursor=AAAFXwAAATsAAAAAAAAAADFleUp6WldGeVkyaEJablJsY2lJNld5SXdJaXdpTXpJd01EVTNOREV0TlRJd055MDJPREl3TFRJMk16UXROREkzTkdJMk5ESTVaVEZoSWl3aU1qQXlOQ0pkZlE9PQ==',
        f'{DOMAIN}/fumbles/2024/REG/all/defensiveforcedfumble/DESC?aftercursor=AAAFeAAAATsAAAAAAAAAADFleUp6WldGeVkyaEJablJsY2lJNld5SXdJaXdpTXpJd01EVTNORGd0TkRrMk1pMHlOek14TFRGa09USXRZMlU0TUdRMU1USmtZak0wSWl3aU1qQXlOQ0pkZlE9PQ==',
        f'{DOMAIN}/fumbles/2024/REG/all/defensiveforcedfumble/DESC?aftercursor=AAAFkQAAATsAAAAAAAAAADFleUp6WldGeVkyaEJablJsY2lJNld5SXdJaXdpTXpJd01EVTNORGt0TkdNMU9TMDNNek14TFdReVpUa3RaRFJsTURsaVpUSTRZekk0SWl3aU1qQXlOQ0pkZlE9PQ==',
        f'{DOMAIN}/fumbles/2024/REG/all/defensiveforcedfumble/DESC?aftercursor=AAAFqgAAATsAAAAAAAAAADFleUp6WldGeVkyaEJablJsY2lJNld5SXdJaXdpTXpJd01EVTNOR1l0TkdZMU1TMDJNelU1TFRneU4yRXRNV1prWlRoa1l6RTNPVGd5SWl3aU1qQXlOQ0pkZlE9PQ=='
]

'''
f'{DOMAIN}/tackles/2024/post/all/defensivecombinetackles/desc',
        f'{DOMAIN}/tackles/2024/REG/all/defensivecombinetackles/DESC?aftercursor=AAAAGQAAABlAXoAAAAAAADFleUp6WldGeVkyaEJablJsY2lJNld5SXhNakl1TUNJc0lqTXlNREEwTkRReExUVTJNREl0TmpZMU5DMWxaR0k0TFdGaE5tUmhOVFkzTXpJMU15SXNJakl3TWpRaVhYMD0=',
        f'{DOMAIN}/tackles/2024/REG/all/defensivecombinetackles/DESC?aftercursor=AAAAMgAAADJAWcAAAAAAADFleUp6WldGeVkyaEJablJsY2lJNld5SXhNRE11TUNJc0lqTXlNREEwWkRRMUxUVTBNemd0TXpRME1pMHlabU5pTFRrMVltWTVOV0kxTWpVNE1TSXNJakl3TWpRaVhYMD0=',
        f'{DOMAIN}/tackles/2024/REG/all/defensivecombinetackles/DESC?aftercursor=AAAASwAAAEpAVgAAAAAAADFleUp6WldGeVkyaEJablJsY2lJNld5STRPQzR3SWl3aU16SXdNRFJrTkRNdE5HSTFOQzA0TnpZMUxXRTJaRGt0TlRWallUQTFOVEF5WlRaaUlpd2lNakF5TkNKZGZRPT0=',
        f'{DOMAIN}/tackles/2024/REG/all/defensivecombinetackles/DESC?aftercursor=AAAAZAAAAGJAVAAAAAAAADFleUp6WldGeVkyaEJablJsY2lJNld5STRNQzR3SWl3aU16SXdNRFJrTkRrdE5HTTFPQzA0T1RjeUxUTmpZMkl0TWpJeFlqUmhNV015TkRSbUlpd2lNakF5TkNKZGZRPT0=',
        f'{DOMAIN}/tackles/2024/REG/all/defensivecombinetackles/DESC?aftercursor=AAAAfQAAAHdAUsAAAAAAADFleUp6WldGeVkyaEJablJsY2lJNld5STNOUzR3SWl3aU16SXdNRFUzTkRrdE5HTTNOQzAzTVRZNUxXWXlOVEV0WlRZNVl6UXdaRFExTnpZMUlpd2lNakF5TkNKZGZRPT0=',
        f'{DOMAIN}/tackles/2024/REG/all/defensivecombinetackles/DESC?aftercursor=AAAAlgAAAJVAUUAAAAAAADFleUp6WldGeVkyaEJablJsY2lJNld5STJPUzR3SWl3aU16SXdNRFJoTkdZdE5HVTBNQzB6T1RNMExUTXhZVFV0TWprd1pEQmtaV1l4WVRSaUlpd2lNakF5TkNKZGZRPT0=',
        f'{DOMAIN}/tackles/2024/REG/all/defensivecombinetackles/DESC?aftercursor=AAAArwAAAKxATwAAAAAAADFleUp6WldGeVkyaEJablJsY2lJNld5STJNaTR3SWl3aU16SXdNRFV6TkRrdE5HUXlOQzA0TmpneExXVTNaVGt0TVRoalpqSmhPRFZpT0dJNUlpd2lNakF5TkNKZGZRPT0=',
        f'{DOMAIN}/tackles/2024/REG/all/defensivecombinetackles/DESC?aftercursor=AAAAxwAAAMZATQAAAAAAADFleUp6WldGeVkyaEJablJsY2lJNld5STFPQzR3SWl3aU16SXdNRFF6TkdZdE5HWTJNUzAzTVRNNUxUaG1aVGd0TXpjeE9UZGxZVFkzTUdaaElpd2lNakF5TkNKZGZRPT0=',
        f'{DOMAIN}/tackles/2024/REG/all/defensivecombinetackles/DESC?aftercursor=AAAA4AAAAN9ASwAAAAAAADFleUp6WldGeVkyaEJablJsY2lJNld5STFOQzR3SWl3aU16SXdNRFEwTkRrdE5ERXdOUzB4T1RFeExUVTJNbU10TjJRM05tWTBaVFE1WWprNUlpd2lNakF5TkNKZGZRPT0=',
        f'{DOMAIN}/tackles/2024/REG/all/defensivecombinetackles/DESC?aftercursor=AAAA-QAAAPNASYAAAAAAADFleUp6WldGeVkyaEJablJsY2lJNld5STFNUzR3SWl3aU16SXdNRFV6TlRjdE5EVXdNeTAyTVRreExUTXhNakV0TVRFeFlqVXhaV1ExWkRGaElpd2lNakF5TkNKZGZRPT0=',
        f'{DOMAIN}/tackles/2024/REG/all/defensivecombinetackles/DESC?aftercursor=AAABDwAAAQ1AR4AAAAAAADFleUp6WldGeVkyaEJablJsY2lJNld5STBOeTR3SWl3aU16SXdNRFEzTkRFdE5USTFNaTA0T0RrMkxUSmpNV1F0TldGaFptVmpOMlEwTjJVNElpd2lNakF5TkNKZGZRPT0=',
        f'{DOMAIN}/tackles/2024/REG/all/defensivecombinetackles/DESC?aftercursor=AAABKAAAASNARoAAAAAAADFleUp6WldGeVkyaEJablJsY2lJNld5STBOUzR3SWl3aU16SXdNRFJpTkdZdE5EZzBOaTB4TkRNMUxUTXhPVEF0TUdVMllUaGpaR1JsTjJFd0lpd2lNakF5TkNKZGZRPT0=',
        f'{DOMAIN}/tackles/2024/REG/all/defensivecombinetackles/DESC?aftercursor=AAABPgAAAT1ARQAAAAAAADFleUp6WldGeVkyaEJablJsY2lJNld5STBNaTR3SWl3aU16SXdNRFEwTkRrdE5EY3lNQzB5T0RRMUxXWTJPV1V0TWpBeE1EWTBPVGxtTm1ZMklpd2lNakF5TkNKZGZRPT0=',
        f'{DOMAIN}/tackles/2024/REG/all/defensivecombinetackles/DESC?aftercursor=AAABUwAAAUtARIAAAAAAADFleUp6WldGeVkyaEJablJsY2lJNld5STBNUzR3SWl3aU16SXdNRFV3TkRFdE5Ua3hOaTA0T1RnMkxXUTVabUl0WkRWallqRmtNVGRsWWpReUlpd2lNakF5TkNKZGZRPT0=',
        f'{DOMAIN}/tackles/2024/REG/all/defensivecombinetackles/DESC?aftercursor=AAABaQAAAWdAQ4AAAAAAADFleUp6WldGeVkyaEJablJsY2lJNld5SXpPUzR3SWl3aU16SXdNRFExTlRBdE5EVTBNUzAxTWpreExUa3pOamt0WWpsa01HTXhOekJoTlRZMUlpd2lNakF5TkNKZGZRPT0=',
        f'{DOMAIN}/tackles/2024/REG/all/defensivecombinetackles/DESC?aftercursor=AAABfQAAAXdAQoAAAAAAADFleUp6WldGeVkyaEJablJsY2lJNld5SXpOeTR3SWl3aU16SXdNRFJoTkdZdE5HVXhOaTB5TlRrMUxUUmhNV1F0T0Rsa1lUQXlZMlV3TkRsaUlpd2lNakF5TkNKZGZRPT0=',
        f'{DOMAIN}/tackles/2024/REG/all/defensivecombinetackles/DESC?aftercursor=AAABkgAAAYhAQYAAAAAAADFleUp6WldGeVkyaEJablJsY2lJNld5SXpOUzR3SWl3aU16SXdNRFZpTkdVdE5HWXdNQzB3T0RFd0xUUTBPR1V0TURRd05qQmpOelprTWpRMklpd2lNakF5TkNKZGZRPT0=',
        f'{DOMAIN}/tackles/2024/REG/all/defensivecombinetackles/DESC?aftercursor=AAABqQAAAaFAQAAAAAAAADFleUp6WldGeVkyaEJablJsY2lJNld5SXpNaTR3SWl3aU16SXdNRFUwTkRFdE5Ua3hOQzAyTVRrekxUWTVPRGd0WkdRME5ESmtZbVkzWmpZMUlpd2lNakF5TkNKZGZRPT0=',
        f'{DOMAIN}/tackles/2024/REG/all/defensivecombinetackles/DESC?aftercursor=AAABvwAAAb5APQAAAAAAADFleUp6WldGeVkyaEJablJsY2lJNld5SXlPUzR3SWl3aU16SXdNRFF6TkdZdE5HTXlNUzAwTXprM0xXWTNOVGN0TkRabVlqazFaRFl6WmpWbElpd2lNakF5TkNKZGZRPT0=',
        f'{DOMAIN}/tackles/2024/REG/all/defensivecombinetackles/DESC?aftercursor=AAAB1wAAAcpAPAAAAAAAADFleUp6WldGeVkyaEJablJsY2lJNld5SXlPQzR3SWl3aU16SXdNRFV3TkRFdE5UQTNNaTB6TVRjMExXRXdaakl0Tm1Jek5UVmtOamN5TnpZd0lpd2lNakF5TkNKZGZRPT0=',
        f'{DOMAIN}/tackles/2024/REG/all/defensivecombinetackles/DESC?aftercursor=AAAB7wAAAeZAOgAAAAAAADFleUp6WldGeVkyaEJablJsY2lJNld5SXlOaTR3SWl3aU16SXdNRFE0TkRFdE5USTJOQzAzTmpBeExUSmpabVV0WkRNek5HTTVNR1UwTkRGbUlpd2lNakF5TkNKZGZRPT0=',
        f'{DOMAIN}/tackles/2024/REG/all/defensivecombinetackles/DESC?aftercursor=AAACBwAAAf5AOAAAAAAAADFleUp6WldGeVkyaEJablJsY2lJNld5SXlOQzR3SWl3aU16SXdNRFJrTkRrdE5UUXhOQzA1TnpRNUxUVXlOall0WVdVM05UZzFNak15TlRRd0lpd2lNakF5TkNKZGZRPT0=',
        f'{DOMAIN}/tackles/2024/REG/all/defensivecombinetackles/DESC?aftercursor=AAACIAAAAhpANgAAAAAAADFleUp6WldGeVkyaEJablJsY2lJNld5SXlNaTR3SWl3aU16SXdNRFE0TkRVdE5USXdOQzB5TVRBM0xUazRPV0l0WWpsaE1HVmlaakV3T0RRMUlpd2lNakF5TkNKZGZRPT0=',
        f'{DOMAIN}/tackles/2024/REG/all/defensivecombinetackles/DESC?aftercursor=AAACOQAAAjVANAAAAAAAADFleUp6WldGeVkyaEJablJsY2lJNld5SXlNQzR3SWl3aU16SXdNRFJrTkRFdE5EUXpOQzAzTlRnMUxURmxaVEF0TURNNVlqVmhPRFl6Wm1RNUlpd2lNakF5TkNKZGZRPT0=',
        f'{DOMAIN}/tackles/2024/REG/all/defensivecombinetackles/DESC?aftercursor=AAACUQAAAkJAMwAAAAAAADFleUp6WldGeVkyaEJablJsY2lJNld5SXhPUzR3SWl3aU16SXdNRFUzTkRndE5Ea3pPUzA1T1RneUxXUmhabVV0T0dGaFkyVTBZbUV3WkRRNElpd2lNakF5TkNKZGZRPT0=',
        f'{DOMAIN}/tackles/2024/REG/all/defensivecombinetackles/DESC?aftercursor=AAACagAAAl1AMQAAAAAAADFleUp6WldGeVkyaEJablJsY2lJNld5SXhOeTR3SWl3aU16SXdNRFJrTkRrdE5HTTFPUzAzTmpVMExUZzJNRGd0TmpJeU1EVXhZalV6WW1Reklpd2lNakF5TkNKZGZRPT0=',
        f'{DOMAIN}/tackles/2024/REG/all/defensivecombinetackles/DESC?aftercursor=AAACgwAAAn9ALgAAAAAAADFleUp6WldGeVkyaEJablJsY2lJNld5SXhOUzR3SWl3aU16SXdNRFJoTkdZdE5HVTBOQzA1TnpFMUxXUmpPRE10WmpWbFlqYzNZelF6WlRWaklpd2lNakF5TkNKZGZRPT0=',
        f'{DOMAIN}/tackles/2024/REG/all/defensivecombinetackles/DESC?aftercursor=AAACnAAAAo5ALAAAAAAAADFleUp6WldGeVkyaEJablJsY2lJNld5SXhOQzR3SWl3aU16SXdNRFJoTkRVdE5EWXpOUzB3TXprNUxXUmlabUl0TXpjM05XRTJZV00zT1RWaklpd2lNakF5TkNKZGZRPT0=',
        f'{DOMAIN}/tackles/2024/REG/all/defensivecombinetackles/DESC?aftercursor=AAACtQAAAqdAKgAAAAAAADFleUp6WldGeVkyaEJablJsY2lJNld5SXhNeTR3SWl3aU16SXdNRFJsTkdFdE5HWXpOaTAzTWpFMExXVmhNMkV0TlRoa1lXRTFaak5sWTJJMUlpd2lNakF5TkNKZGZRPT0=',
        f'{DOMAIN}/tackles/2024/REG/all/defensivecombinetackles/DESC?aftercursor=AAACzgAAAr5AKAAAAAAAADFleUp6WldGeVkyaEJablJsY2lJNld5SXhNaTR3SWl3aU16SXdNRFJoTkdZdE5EZ3lNQzB5TVRnNUxUWmxNall0TVRBM05tSTBPVGRqTVdReUlpd2lNakF5TkNKZGZRPT0=',
        f'{DOMAIN}/tackles/2024/REG/all/defensivecombinetackles/DESC?aftercursor=AAAC5wAAAtlAJgAAAAAAADFleUp6WldGeVkyaEJablJsY2lJNld5SXhNUzR3SWl3aU16SXdNRFV5TkRrdE5HTXhOQzAyTmpRNExUSXhORFF0TUdVeVpUVmhZbVExTURJd0lpd2lNakF5TkNKZGZRPT0=',
        f'{DOMAIN}/tackles/2024/REG/all/defensivecombinetackles/DESC?aftercursor=AAADAAAAAv1AIgAAAAAAADFleUp6WldGeVkyaEJablJsY2lJNld5STVMakFpTENJek1qQXdOREkxTlMwMU5EUTVMVEExTWpBdE9UWTNaUzAxTmpjM09UWTBOVEJpTWpnaUxDSXlNREkwSWwxOQ==',
        f'{DOMAIN}/tackles/2024/REG/all/defensivecombinetackles/DESC?aftercursor=AAADGQAAAxVAIAAAAAAAADFleUp6WldGeVkyaEJablJsY2lJNld5STRMakFpTENJek1qQXdORGMwTVMwMU9UVTBMVFU0TnpjdE5qUmpNUzB3TjJFM1lqZGhPRFpsT0dZaUxDSXlNREkwSWwxOQ==',
        f'{DOMAIN}/tackles/2024/REG/all/defensivecombinetackles/DESC?aftercursor=AAADMgAAAyxAHAAAAAAAADFleUp6WldGeVkyaEJablJsY2lJNld5STNMakFpTENJek1qQXdOREkxTWkwMFpqWTVMVFV3TURNdFpUY3lZaTAxTURKaFpUbGhZVEUxTlRnaUxDSXlNREkwSWwxOQ==',
        f'{DOMAIN}/tackles/2024/REG/all/defensivecombinetackles/DESC?aftercursor=AAADSwAAA0tAGAAAAAAAADFleUp6WldGeVkyaEJablJsY2lJNld5STJMakFpTENJek1qQXdOREUwWXkwMFl6Y3dMVEkxT0RJdE5HTmpZUzB4WVRJd056QmpPV0V3T1RBaUxDSXlNREkwSWwxOQ==',
        f'{DOMAIN}/tackles/2024/REG/all/defensivecombinetackles/DESC?aftercursor=AAADZAAAA0tAGAAAAAAAADFleUp6WldGeVkyaEJablJsY2lJNld5STJMakFpTENJek1qQXdOVE0wWkMwME9UZ3lMVFkxTURNdFlUTTVNaTFrWXpBME56aGtPR0pqWWpNaUxDSXlNREkwSWwxOQ==',
        f'{DOMAIN}/tackles/2024/REG/all/defensivecombinetackles/DESC?aftercursor=AAADfQAAA2tAFAAAAAAAADFleUp6WldGeVkyaEJablJsY2lJNld5STFMakFpTENJek1qQXdORGcwTVMwMU1qQTBMVGt5T1RjdFl6YzJOQzB4Tnpnd1lqQTBPRGRqT1RNaUxDSXlNREkwSWwxOQ==',
        f'{DOMAIN}/tackles/2024/REG/all/defensivecombinetackles/DESC?aftercursor=AAADlgAAA5VAEAAAAAAAADFleUp6WldGeVkyaEJablJsY2lJNld5STBMakFpTENJek1qQXdOREUwTkMwME1UUTVMVGt5TlRNdE5ERXpZUzA0TW1aa05EUTNOalZtWWpnaUxDSXlNREkwSWwxOQ==',
        f'{DOMAIN}/tackles/2024/REG/all/defensivecombinetackles/DESC?aftercursor=AAADrwAAA5VAEAAAAAAAADFleUp6WldGeVkyaEJablJsY2lJNld5STBMakFpTENJek1qQXdOR0UwWmkwMFpUVXhMVGd6TXpVdE4yUTJZUzFsTm1JM1ptWXlaVEE1WldNaUxDSXlNREkwSWwxOQ==',
        f'{DOMAIN}/tackles/2024/REG/all/defensivecombinetackles/DESC?aftercursor=AAADyAAAA5VAEAAAAAAAADFleUp6WldGeVkyaEJablJsY2lJNld5STBMakFpTENJek1qQXdOVGMwT0MwME9UZ3dMVGMyT0RRdFpEZ3hZaTFrTkdaa09ERmlNRFJpWTJJaUxDSXlNREkwSWwxOQ==',
        f'{DOMAIN}/tackles/2024/REG/all/defensivecombinetackles/DESC?aftercursor=AAAD4QAAA8xACAAAAAAAADFleUp6WldGeVkyaEJablJsY2lJNld5SXpMakFpTENJek1qQXdORE0wWmkwMFpUUTJMVEU1T1RrdE5UbGxPUzB5TlROalpXUmxaV016WVRVaUxDSXlNREkwSWwxOQ==',
        f'{DOMAIN}/tackles/2024/REG/all/defensivecombinetackles/DESC?aftercursor=AAAD-gAAA8xACAAAAAAAADFleUp6WldGeVkyaEJablJsY2lJNld5SXpMakFpTENJek1qQXdOR1EwTVMwMU5ERTVMVFUyTkRJdE5qWXpOaTB4WlRSall6SXhaVGMwTW1RaUxDSXlNREkwSWwxOQ==',
        f'{DOMAIN}/tackles/2024/REG/all/defensivecombinetackles/DESC?aftercursor=AAAEEwAAA8xACAAAAAAAADFleUp6WldGeVkyaEJablJsY2lJNld5SXpMakFpTENJek1qQXdOVFEwTVMwMU9UWXhMVEExTlRNdE9URTNZeTB6WVRVMlpUVmhOR015Tm1NaUxDSXlNREkwSWwxOQ==',
        f'{DOMAIN}/tackles/2024/REG/all/defensivecombinetackles/DESC?aftercursor=AAAELAAABCVAAAAAAAAAADFleUp6WldGeVkyaEJablJsY2lJNld5SXlMakFpTENJek1qQXdOREkwTVMwMU5EVTBMVFV3TmpVdFpHSmlPUzA0TlRReFpUZGtaalExT0dFaUxDSXlNREkwSWwxOQ==',
        f'{DOMAIN}/tackles/2024/REG/all/defensivecombinetackles/DESC?aftercursor=AAAERQAABCVAAAAAAAAAADFleUp6WldGeVkyaEJablJsY2lJNld5SXlMakFpTENJek1qQXdORFUxTWkwMU5EZ3dMVEE1TWpBdE16SmhOaTB4TmpWaU5XRTNOemd4TkdVaUxDSXlNREkwSWwxOQ==',
        f'{DOMAIN}/tackles/2024/REG/all/defensivecombinetackles/DESC?aftercursor=AAAEXgAABCVAAAAAAAAAADFleUp6WldGeVkyaEJablJsY2lJNld5SXlMakFpTENJek1qQXdORGswWlMwME56TTRMVEkyTmpFdFltRTNOeTA0WXpNMU5EUTROakk1TXpnaUxDSXlNREkwSWwxOQ==',
        f'{DOMAIN}/tackles/2024/REG/all/defensivecombinetackles/DESC?aftercursor=AAAEdwAABCVAAAAAAAAAADFleUp6WldGeVkyaEJablJsY2lJNld5SXlMakFpTENJek1qQXdOR1kwWXkwME1UWTFMVGt6TWpVdFl6STFPUzB3TVdFMU1qRXlNREpsTW1ZaUxDSXlNREkwSWwxOQ==',
        f'{DOMAIN}/tackles/2024/REG/all/defensivecombinetackles/DESC?aftercursor=AAAEkAAABCVAAAAAAAAAADFleUp6WldGeVkyaEJablJsY2lJNld5SXlMakFpTENJek1qQXdOVE0xTlMwMFpURXpMVEUyT0RBdFkyUTNaUzB5TURsbVptUmhOamMyTlRNaUxDSXlNREkwSWwxOQ==',
        f'{DOMAIN}/tackles/2024/REG/all/defensivecombinetackles/DESC?aftercursor=AAAEqQAABKM_8AAAAAAAADFleUp6WldGeVkyaEJablJsY2lJNld5SXhMakFpTENJek1qQXdOREUwWXkwMFl6QTJMVE00TVRZdFlUazVOUzB5TVRZeE5HSTVPRFEzT1RnaUxDSXlNREkwSWwxOQ==',
        f'{DOMAIN}/tackles/2024/REG/all/defensivecombinetackles/DESC?aftercursor=AAAEwgAABKM_8AAAAAAAADFleUp6WldGeVkyaEJablJsY2lJNld5SXhMakFpTENJek1qQXdOREkxTWkwMFpqRXpMVEk1T0RndE56QXhZaTA1TUdJMVlqQTVOak0yWTJFaUxDSXlNREkwSWwxOQ==',
        f'{DOMAIN}/tackles/2024/REG/all/defensivecombinetackles/DESC?aftercursor=AAAE2wAABKM_8AAAAAAAADFleUp6WldGeVkyaEJablJsY2lJNld5SXhMakFpTENJek1qQXdORFEwT1MwME56SXhMVGc0T1RVdFptTXlNeTFrTW1KbU5HTTVNakZpWkRRaUxDSXlNREkwSWwxOQ==',
        f'{DOMAIN}/tackles/2024/REG/all/defensivecombinetackles/DESC?aftercursor=AAAE9AAABKM_8AAAAAAAADFleUp6WldGeVkyaEJablJsY2lJNld5SXhMakFpTENJek1qQXdORGMwT1MwMFl6STNMVEl4T0RFdE1XUmxaQzFrWkRRNU5qUTRNMkUwTUdRaUxDSXlNREkwSWwxOQ==',
        f'{DOMAIN}/tackles/2024/REG/all/defensivecombinetackles/DESC?aftercursor=AAAFDQAABKM_8AAAAAAAADFleUp6WldGeVkyaEJablJsY2lJNld5SXhMakFpTENJek1qQXdORGcwWmkwMU1ERTJMVEUxTURBdFltWmlaUzFqT0RkaU1qa3lZall6T1dNaUxDSXlNREkwSWwxOQ==',
        f'{DOMAIN}/tackles/2024/REG/all/defensivecombinetackles/DESC?aftercursor=AAAFJgAABKM_8AAAAAAAADFleUp6WldGeVkyaEJablJsY2lJNld5SXhMakFpTENJek1qQXdOR0kwTVMwMFpESTBMVFF3TlRndE5XRXpZaTB3TVdKa016bGxObUV5TldRaUxDSXlNREkwSWwxOQ==',
        f'{DOMAIN}/tackles/2024/REG/all/defensivecombinetackles/DESC?aftercursor=AAAFPwAABKM_8AAAAAAAADFleUp6WldGeVkyaEJablJsY2lJNld5SXhMakFpTENJek1qQXdOR1EwTXkwME16QTFMVGMyT0RrdE9Ea3laQzB3TlRabVltUXpOVEk0TVdRaUxDSXlNREkwSWwxOQ==',
        f'{DOMAIN}/tackles/2024/REG/all/defensivecombinetackles/DESC?aftercursor=AAAFWAAABKM_8AAAAAAAADFleUp6WldGeVkyaEJablJsY2lJNld5SXhMakFpTENJek1qQXdOR1kxTWkwMVlUUXhMVFV5T1RFdFltVmlPQzFsWW1ZME1Ua3dNekJoTVRnaUxDSXlNREkwSWwxOQ==',
        f'{DOMAIN}/tackles/2024/REG/all/defensivecombinetackles/DESC?aftercursor=AAAFcQAABKM_8AAAAAAAADFleUp6WldGeVkyaEJablJsY2lJNld5SXhMakFpTENJek1qQXdOVEkwWmkwME1qSTBMVEF3T0RndFlqQmlPUzB5WTJNd05UUmlZVGN4TjJRaUxDSXlNREkwSWwxOQ==',
        f'{DOMAIN}/tackles/2024/REG/all/defensivecombinetackles/DESC?aftercursor=AAAFigAABKM_8AAAAAAAADFleUp6WldGeVkyaEJablJsY2lJNld5SXhMakFpTENJek1qQXdOVE0xTkMwME9UWXlMVGd3TmpjdFlURmxOQzAyWkRobVkyVTNNak0xWVRnaUxDSXlNREkwSWwxOQ==',
        f'{DOMAIN}/tackles/2024/REG/all/defensivecombinetackles/DESC?aftercursor=AAAFowAABKM_8AAAAAAAADFleUp6WldGeVkyaEJablJsY2lJNld5SXhMakFpTENJek1qQXdOVGMwT1MwMFl6SXdMVFF6T1RNdE9UY3hOUzFrWldFME1tVmlNalJrWXpZaUxDSXlNREkwSWwxOQ==',
        f'{DOMAIN}/interceptions/2024/post/all/defensiveinterceptions/desc',
        f'{DOMAIN}/interceptions/2024/REG/all/defensiveinterceptions/DESC?aftercursor=AAAAGQAAABNACAAAAAAAADFleUp6WldGeVkyaEJablJsY2lJNld5SXpJaXdpTXpJd01EUTBOR1l0TkRRek5DMHhORFl3TFdJek56Z3RPVEV4Tnpaa056QTBZemMzSWl3aU1qQXlOQ0pkZlE9PQ==',
        f'{DOMAIN}/interceptions/2024/REG/all/defensiveinterceptions/DESC?aftercursor=AAAAMgAAACZAAAAAAAAAADFleUp6WldGeVkyaEJablJsY2lJNld5SXlJaXdpTXpJd01EUTNOVEl0TkRreU9TMDJOVFkxTFRkbU5UUXROVGxtWm1NM016VTNNak5tSWl3aU1qQXlOQ0pkZlE9PQ==',
        f'{DOMAIN}/interceptions/2024/REG/all/defensiveinterceptions/DESC?aftercursor=AAAASwAAACZAAAAAAAAAADFleUp6WldGeVkyaEJablJsY2lJNld5SXlJaXdpTXpJd01EVXpORGt0TkdReU5DMDROamd4TFdVM1pUa3RNVGhqWmpKaE9EVmlPR0k1SWl3aU1qQXlOQ0pkZlE9PQ==',
        f'{DOMAIN}/interceptions/2024/REG/all/defensiveinterceptions/DESC?aftercursor=AAAAZAAAAFg_8AAAAAAAADFleUp6WldGeVkyaEJablJsY2lJNld5SXhJaXdpTXpJd01EUXlOR1l0TkdNMk55MDRNekF4TFdOa05qWXRNbUkzWlRRMU5ETTVNakpoSWl3aU1qQXlOQ0pkZlE9PQ==',
        f'{DOMAIN}/interceptions/2024/REG/all/defensiveinterceptions/DESC?aftercursor=AAAAfQAAAFg_8AAAAAAAADFleUp6WldGeVkyaEJablJsY2lJNld5SXhJaXdpTXpJd01EUTFOR010TkdNM05DMDRNRFUzTFRnek5UZ3RNR013WWpGak1qaGpPR1F4SWl3aU1qQXlOQ0pkZlE9PQ==',
        f'{DOMAIN}/interceptions/2024/REG/all/defensiveinterceptions/DESC?aftercursor=AAAAlgAAAFg_8AAAAAAAADFleUp6WldGeVkyaEJablJsY2lJNld5SXhJaXdpTXpJd01EUTROR1l0TlRJeE1pMHpOVGsxTFdWaFltWXRObVkyWmpOalpHUmtZelF3SWl3aU1qQXlOQ0pkZlE9PQ==',
        f'{DOMAIN}/interceptions/2024/REG/all/defensiveinterceptions/DESC?aftercursor=AAAArwAAAFg_8AAAAAAAADFleUp6WldGeVkyaEJablJsY2lJNld5SXhJaXdpTXpJd01EUmtOVFV0TlRJMU5pMHpORGt3TFRVM05HRXRNV0k0WVRnMllXRTJPVEZtSWl3aU1qQXlOQ0pkZlE9PQ==',
        f'{DOMAIN}/interceptions/2024/REG/all/defensiveinterceptions/DESC?aftercursor=AAAAyAAAAFg_8AAAAAAAADFleUp6WldGeVkyaEJablJsY2lJNld5SXhJaXdpTXpJd01EVXlOR1l0TldFd05DMDVPRE0wTFRFNU5tRXROelEzTlRNM1pHRm1aak16SWl3aU1qQXlOQ0pkZlE9PQ==',
        f'{DOMAIN}/interceptions/2024/REG/all/defensiveinterceptions/DESC?aftercursor=AAAA4QAAAFg_8AAAAAAAADFleUp6WldGeVkyaEJablJsY2lJNld5SXhJaXdpTXpJd01EVTNOR1l0TlRJMk1TMDJOREV3TFdRNE9UVXRPVFF3WldaaVlXUTJNakpqSWl3aU1qQXlOQ0pkZlE9PQ=='
'''

HEADERS = {'Uer-Agent': 'Mozilla/5.0'}

In [3]:
def get_page_content(url, headers):
  response = requests.get(url, headers=headers)
  soup = BeautifulSoup(response.text, 'html.parser')
  return soup

In [4]:
# Lista para armazenar as URLs dos jogadores e das imagens corrigidas
url_players = []

# Loop sobre todas as URLs principais
for url in URLs:
    # Obter o conteúdo da página para a URL atual
    soup = get_page_content(url, HEADERS)

    # Localizar a tabela de jogadores na página
    table = soup.find('table', {'class': 'd3-o-table d3-o-table--detailed d3-o-player-stats--detailed d3-o-table--sortable'})

    # Iterar sobre todas as linhas que contêm os links para os jogadores
    for row in table.find_all('a', {'class': 'd3-o-player-fullname nfl-o-cta--link'}):
        # Adicionar o link de cada jogador à lista
        url_players.append(row['href'])

In [ ]:
url_players = pd.unique(pd.Series(url_players))

In [ ]:
# Configuração das categorias e seus respectivos campos
categories_config = {
    'Fumbles': [
        'team', 'games', 'fumbles', 'fumbles_lost', 'forced_fumbles',
        'own_fumbles_recoveries', 'opponent_fumbles_recoveries', 'touchdowns'
    ]
}

# Diretório para salvar os arquivos
output_dir = "player_stats"
os.makedirs(output_dir, exist_ok=True)

# Função para salvar dados em arquivos CSV
def save_to_csv(category, fields, data):
    file_path = os.path.join(output_dir, f"{category}.csv")
    file_exists = os.path.isfile(file_path)

    fields_with_meta = ['player_id', 'player_name'] + fields

    existing_data = set()
    if file_exists:
        with open(file_path, mode='r', encoding='utf-8') as file:
            reader = csv.DictReader(file)
            for row in reader:
                unique_key = tuple(row.get(field, '') for field in fields_with_meta)
                existing_data.add(unique_key)

    new_data = []
    for row in data:
        unique_key = tuple(row[field] for field in fields_with_meta)
        if unique_key not in existing_data:
            new_data.append(row)

    if new_data:
        with open(file_path, mode='a', newline='', encoding='utf-8') as file:
            writer = csv.DictWriter(file, fieldnames=fields_with_meta)
            if not file_exists:
                writer.writeheader()
            writer.writerows(new_data)

# Função para salvar os dados dos jogadores em um arquivo CSV
def save_players_to_csv(player):
    players_file_path = os.path.join(output_dir, "Players.csv")
    players_file_exists = os.path.isfile(players_file_path)

    # Campos para o arquivo Players.csv
    player_fields = ['player_id', 'player_name', 'player_position', 'team', 'player_image_url']

    # Carregar dados existentes, se o arquivo já existir
    existing_players = set()
    if players_file_exists:
        with open(players_file_path, mode='r', encoding='utf-8') as file:
            reader = csv.DictReader(file)
            for row in reader:
                unique_key = tuple(row[field] for field in player_fields)
                existing_players.add(unique_key)

    # Verificar se o jogador já existe
    unique_key = tuple(player[field] for field in player_fields)
    if unique_key not in existing_players:
        with open(players_file_path, mode='a', newline='', encoding='utf-8') as file:
            writer = csv.DictWriter(file, fieldnames=player_fields)
            if not players_file_exists:
                writer.writeheader()
            writer.writerow(player)

def extract_stats(content, category, fields, player_id, player_name):
    stats_list = []

    if category in content.text:
        table = content.find_next('table')
        rows = table.find_all('tr')

        for row in rows:
            if '2024' in row.text:
                infos = row.find_all('td')
                if len(infos) > len(fields):
                    stats = {field: infos[i + 1].text.strip() if i + 1 < len(infos) else '' for i, field in enumerate(fields)}
                    stats['player_id'] = player_id
                    stats['player_name'] = player_name
                    stats_list.append(stats)

    return stats_list

# Loop principal para processar as URLs dos jogadores
counter = 0
for url in url_players:
    player_id = url.split('/players/')[1].strip('/')
    soup = get_page_content(f'https://www.nfl.com{url}stats/career', HEADERS)
    player_name = soup.find('h1', {'class': 'nfl-c-player-header__title'}).text.strip()

    # Extrair a posição do jogador
    position_tag = soup.find('span', {'class': 'nfl-c-player-header__position'})
    player_position = position_tag.text.strip() if position_tag else 'Unknown'

    # Extrair o time do jogador
    team_tag = soup.find_all('td')
    player_team = team_tag[1].text.strip() if team_tag else 'Unknown'

    # Extrair a URL da imagem do jogador
    img_tag = soup.find('img', {'class': 'img-responsive'})
    if img_tag and 'src' in img_tag.attrs:
        player_image_url = img_tag['src'].replace("t_lazy", "t_player_profile_landscape")
    else:
        player_image_url = 'Unknown'

    counter += 1
    print(counter, '-', player_id)

    # Salvar o jogador em Players.csv
    player_data = {
        'player_id': player_id,
        'player_name': player_name,
        'player_position': player_position,
        'team': player_team,
        'player_image_url': player_image_url
    }
    save_players_to_csv(player_data)

    # Processar estatísticas
    div_table = soup.find_all('div', {'class': 'nfl-o-roster'})

    for content in div_table:
        for category, fields in categories_config.items():
            stats = extract_stats(content, category, fields, player_id, player_name)
            if stats:
                save_to_csv(category, fields, stats)


1 - t-j-watt
2 - zack-baun
3 - zaire-franklin
4 - jessie-bates
5 - jonathan-greenard
6 - nick-herbig
7 - fred-warner
8 - quincy-williams
9 - leo-chenal
10 - lavonte-david
11 - myles-garrett
12 - eric-kendricks
13 - nate-landman
14 - laiatu-latu
15 - marte-mapu
16 - jartavius-martin
17 - isaiah-mcguire
18 - keisean-nixon
19 - bobby-okereke
20 - ed-oliver
21 - amik-robertson
22 - gregory-rousseau
23 - tykee-smith
24 - azeez-al-shaair
25 - dorance-armstrong
26 - derek-barnett
27 - christian-benford
28 - nik-bonitto
29 - joey-bosa
30 - sean-murphy-bunting
31 - oren-burks
32 - brian-burns
33 - jalen-carter
34 - zaven-collins
35 - kamren-curl
36 - michael-danna
37 - deshon-elliott
38 - kingsley-enagbare
39 - a-j-epenesa
40 - braden-fiske
41 - dante-fowler
42 - derick-hall
43 - kyle-hamilton
44 - trey-hendrickson
45 - neville-hewitt
46 - amani-hooker
47 - marlon-humphrey
48 - jalyx-hunt
49 - jonathan-jones
50 - levi-jones-2
51 - arden-key
52 - javon-kinlaw
53 - marist-liufau
54 - julian-love


ChunkedEncodingError: ('Connection broken: IncompleteRead(30162 bytes read, 17171 more expected)', IncompleteRead(30162 bytes read, 17171 more expected))